In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [3]:
# llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm=ChatOpenAI(model="gpt-4o-mini")

In [4]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        SystemMessagePromptTemplate.from_template(
            template="""
                Write out the following equation using algebric symbols then solve it. Use the format
                EQUATION:...
                SOLUTION:...
            """
        ),
        HumanMessagePromptTemplate.from_template(
            template="{equation_statement}"
        )
    ]
)

In [6]:
statement="5 times x plus 3 times y equals 7; Negative 3 times x plus 5 times y equals 23"

In [7]:

chain={"equation_statement":RunnablePassthrough()}|prompt|llm|StrOutputParser()

In [9]:
print(chain.invoke(input=statement))

EQUATION: 
1) \( 5x + 3y = 7 \)  
2) \( -3x + 5y = 23 \) 

To solve the system of equations, we can use the method of substitution or elimination. Here, we will use the elimination method.

First, let's multiply the first equation by 3 to align the coefficients for \( y \):

\[
3(5x + 3y) = 3(7)
\]
\[
15x + 9y = 21 \quad \text{(Equation 3)}
\]

Now, multiply the second equation by 5:

\[
5(-3x + 5y) = 5(23)
\]
\[
-15x + 25y = 115 \quad \text{(Equation 4)}
\]

Next, we will add Equation 3 and Equation 4 together:

\[
(15x + 9y) + (-15x + 25y) = 21 + 115
\]
\[
(15x - 15x) + (9y + 25y) = 136
\]
\[
34y = 136
\]

Now, solve for \( y \):

\[
y = \frac{136}{34} = 4
\]

Now that we have \( y \), we can substitute it back into one of the original equations to find \( x \). We can use Equation 1:

\[
5x + 3(4) = 7
\]
\[
5x + 12 = 7
\]
\[
5x = 7 - 12
\]
\[
5x = -5
\]
\[
x = \frac{-5}{5} = -1
\]

Thus, the solution to the system of equations is:

SOLUTION: \( x = -1, y = 4 \)


<h3> Using the Stop Keywords</h3>

In [10]:
chain={"equation_statement":RunnablePassthrough()}|prompt|llm.bind(stop="SOLUTION")|StrOutputParser()

In [11]:
print(chain.invoke(input=statement))

EQUATION:  
1) \( 5x + 3y = 7 \)  
2) \( -3x + 5y = 23 \)  

To solve this system of equations, we can use the substitution or elimination method. Here, I'll use the elimination method.

First, we can multiply the first equation by 3 to align the coefficients of \( y \):

\[
3(5x + 3y) = 3 \times 7 \\
15x + 9y = 21
\]

Now we have:

1) \( 15x + 9y = 21 \)  
2) \( -3x + 5y = 23 \)   

Next, we can multiply the second equation by 3 to align the \( x \) coefficients:

\[
3(-3x + 5y) = 3 \times 23 \\
-9x + 15y = 69
\]

Now, we rewrite the system:

1) \( 15x + 9y = 21 \)  
2) \( -9x + 15y = 69 \)  

Next, we will eliminate \( y \). To do that, we can multiply the first equation by 5 and the second equation by 3:

1) \( 5(15x + 9y) = 5 \times 21 \)  
\(- 75x + 45y = 105\)  

2) \( 3(-9x + 15y) = 3 \times 69 \)  
\(- 27x + 45y = 207\)  

Now we have:

1) \( -75x + 45y = 105 \)  
2) \( -27x + 45y = 207 \)  

Now we can subtract the first equation from the second:

\[
(-27x + 45y) - (-75x + 45y

In [12]:
tools=[
        {
            "type":"function",
            "function": 
                {
                    "name": "solver",
                    "description":"Formulates and Solves an Equation",
                    "parameters":
                        {
                            "type":"object",
                            "properties":
                                {
                                    "equation":
                                        {
                                            "type":"string",
                                            "description":"The Algebric Expression of the Equation"
                                        },
                                    "solution":
                                        {
                                            "type":"string",
                                            "description":"The Solution to the Equation"
                                        }
                                },
                            "required":["equation","solution"]
                        },
                },
        }
    ]

In [13]:
chain={"equation_statement":RunnablePassthrough()}|prompt|llm.bind(tools=tools)

In [14]:
chain.invoke(input=statement).pretty_print()

================================== Ai Message ==================================
Tool Calls:
  solver (call_8HKt47JTVufa9ynBzCvKlwoT)
 Call ID: call_8HKt47JTVufa9ynBzCvKlwoT
  Args:
    equation: 5x + 3y = 7
    solution: x = 2, y = -1
  solver (call_yF0KmKMcU3KdOJM2VvJZGSKB)
 Call ID: call_yF0KmKMcU3KdOJM2VvJZGSKB
  Args:
    equation: -3x + 5y = 23
    solution: x = 2, y = -1
